In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

In [12]:
class NN(nn.Module):
  def __init__(self,input_size,num_classes):
    super(NN,self).__init__()
    self.fc1=nn.Linear(input_size,50)
    self.fc2=nn.Linear(50,num_classes)
  def forward(self,x):
    x=F.relu(self.fc1(x))
    x=self.fc2(x)
    return x
m=NN(784,10)
x=torch.randn(64,784) # 784 is batch size and batch size is 64
print(m(x).shape)

#Set Device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Hyperparameters
input_size=784
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=5
train_dataset=datasets.MNIST(root='data',train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=64,shuffle=True)
test_dataset=datasets.MNIST(root='data',train=False,transform=transforms.ToTensor())
test_loader=DataLoader(dataset=test_dataset, batch_size=64,shuffle=True)

torch.Size([64, 10])


In [21]:
model=NN(input_size=input_size,num_classes=num_classes).to(device)
# Loss Function
criterion=nn.CrossEntropyLoss()
# Optimizer
optimizer=optim.Adam(model.parameters(),lr=learning_rate)
# Train Network
for epoch in range(num_epochs):
  for batch_idx,(data,target) in enumerate(train_loader):
    # Get data to cuda
    data=data.to(device=device)
    target=target.to(device=device)
    data=data.reshape(data.shape[0],-1)
    # Get data to cuda
    #Forward
    scores=model(data)
    loss=criterion(scores,target)
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step() # gradient descent

In [27]:
def accuracy(loader,model):
  c=0
  s=0
  model.eval()
  i=1
  j=1
  with torch.no_grad(): # Dont remember the gradents during training
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)
      if (i==1):
        print(x.shape)
        print(y.shape) # y is batch size
        i+=1
      x=x.reshape(x.shape[0],-1)
      scores=model(x)
      _,p=scores.max(1)
      c+=(y==p).sum() # No of matching for that batch size which is 64
      s+=p.shape[0]  # S is batch size
  return c/s

print(f'Train Accuracy: {accuracy(train_loader,model)*100}')
print(f'Test Accuracy: {accuracy(test_loader,model)*100}')



torch.Size([64, 1, 28, 28])
torch.Size([64])
Train Accuracy: 97.25166320800781
torch.Size([64, 1, 28, 28])
torch.Size([64])
Test Accuracy: 96.58000183105469
